In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

df = pd.read_csv('../data/airplane_df.csv')

## Filter to only US accidents 

In [ ]:
df = df.set_index('Event.Id')

us_accidents = df[df['Country'] == 'United States']

## Group injuries by state 

In [ ]:
injuries_by_state = us_accidents.groupby('State').sum(numeric_only = True).loc[:, 'Total.Fatal.Injuries':]
injuries_by_state['Total.People'] = injuries_by_state.sum(axis=1)

In [ ]:
injuries_by_state

## Quick visualization of injury distribution by state 
Here we can see how in every state, the majority of people were uninjured after an airplane incident.

In [ ]:
plt.style.use('ggplot')
injuries_by_state[injuries_by_state.columns[:4]].plot(kind='barh', stacked=True, figsize=(12, 12),
                                                     ylabel='Number of People', xlabel = 'States');

# Fatality percentage
A column has been added for the fatality percentage by state. This was obtained by dividing the number of total fatal injuries by the total number of people involved in the accidents.

In [ ]:
injuries_by_state['Fatality.Perc'] = (injuries_by_state['Total.Fatal.Injuries'] / injuries_by_state['Total.People']) * 100
injuries_by_state.sort_values(by='Fatality.Perc', ascending=False)

## Total Accidents
Here we have added a column for the total accidents per state.

In [ ]:
accidents_per_state = us_accidents.groupby('State').count()['Accident.Number'].rename('Total.Accidents')
injuries_by_state = injuries_by_state.merge(accidents_per_state, on='State', how='left')
injuries_by_state

## Average Plane Capacity
Here we can see the average plane capacity determined by dividing the total number of people by the total number of accidents.

The table below has been adjusted, so we can only see the states with number of accidents in the 25th percentile. Anything lower than that can provide deceptive information. For example, a place that has only had one very serious accident would show a disproportionately high fatality percentage.

In [ ]:
injuries_by_state['Avg.Plane.Capacity'] = injuries_by_state['Total.People'] / injuries_by_state['Total.Accidents']

In [ ]:
injuries_by_state[['Total.People', 'Fatality.Perc', 'Total.Accidents', 'Avg.Plane.Capacity']].sort_values(by=['Total.Accidents'], ascending=False). \
loc[(injuries_by_state['Total.Accidents'] >= injuries_by_state['Total.Accidents'].quantile(0.25))]

## Fatality percentage vs average plane capacity
We can observe that there's a clear exponential decay in fatalities as the plane capacity increases.

In [ ]:
injuries_by_state.plot(kind='scatter', y='Fatality.Perc', x='Avg.Plane.Capacity');

## Export to CSV. DO NOT RUN the cell below unless an export of 'injuries_by_state' file is needed.

In [ ]:

#injuries_by_state.to_csv('injuries_by_state.csv', encoding='UTF-8')